In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
%%writefile phonebook_search.cu
#include <bits/stdc++.h>

using namespace std;

ifstream phonebook("/content/drive/MyDrive/pralllel-processing-dataset/labtest_dataset2.txt");

string getInput() {
  char ch;
  string str;
  bool reading = false;
  while (phonebook.get(ch)) {
    if (reading) {
      if (ch == '"') {
        break;
      }

      str.push_back(ch);
    } else if (ch == '"') {
      reading = true;
    }
  }
  return str;
}

struct Contact {
  char id[20];
  char name[65];
  char number[15];
};

vector<Contact> prepareContactList() {
  vector<Contact> contactList;
  while (phonebook.peek() != EOF) {
    Contact contact;
    strcpy(contact.id, getInput().c_str());
    strcpy(contact.name, getInput().c_str());
    strcpy(contact.number, getInput().c_str());
    contactList.push_back(contact);
  }
  return contactList;
}

__device__ bool search(char *text, const char *term) {
  int m = 0, n = 0;
  while (text[m]) m++;
  while (term[n]) n++;
  if (m < n) {
    return false;
  }

  for (int i = 0; i < m-n+1; i++) {
    bool matched = true;
    for (int j = 0; j < n; j++) {
      if (text[i+j] != term[j]) {
        matched = false;
        break;
      }
    }
    if (matched) {
      return true;
    }
  }
  return false;
}

__global__ void kernel(char *d_term, Contact *d_contacts, int offset) {
  if (search(d_contacts[threadIdx.x+offset].name, d_term)) {
    printf("Kernel %d %s %s\n", threadIdx.x + offset, d_contacts[threadIdx.x+offset].name, d_contacts[threadIdx.x+offset].number);
  }
}

int main(int argc, char* argv[]) {
  if (argc < 3) {
    cout << "Too few arguments." << endl;
    return 1;
  }
  vector<Contact> contactList = prepareContactList();

  int n = contactList.size();
  cout << n << '\n';
  int threads = atoi(argv[1]);
  string term = "";
  for (int i = 2; i < argc; i++) {
    if (i != 2) {
      term += " ";
    }
    term += argv[i];
  }


  char *d_term;
  cudaMalloc(&d_term, sizeof(term));
  cudaMemcpy(d_term, (char *)term.c_str(), sizeof(term), cudaMemcpyHostToDevice);

  Contact *d_contacts;
  cudaMalloc(&d_contacts, n * sizeof(Contact));
  cudaMemcpy(d_contacts, contactList.data(), n * sizeof(Contact), cudaMemcpyHostToDevice);

  int remaining = n;
  while (remaining) {
    int tLimit = min(threads, remaining);
    kernel<<<1,tLimit>>>(d_term, d_contacts, n-remaining);
    remaining -= tLimit;
  }
  cudaDeviceSynchronize();


  return 0;
}

Overwriting phonebook_search.cu


In [32]:
!nvcc -arch=sm_75 phonebook_search.cu -o phonebook_search

In [33]:
!time ./phonebook_search 4 RIPON ROY > output.txt


real	0m30.454s
user	0m25.827s
sys	0m4.180s


In [35]:
!cat output.txt |

18147953
Kernel 60672 RIPON ROY 013 88 872
Kernel 159542 RIPON ROY 016 58 888
Kernel 682720 RIPON ROY 011 08 714
Kernel 996885 RIPON ROY 011 00 344
Kernel 1003547 RIPON ROY 011 62 125
Kernel 1633638 RIPON ROY 018 34 273
Kernel 1633976 RIPON ROY 017 54 754
Kernel 1664005 RIPON ROY 011 65 733
Kernel 1664074 RIPON ROY 016 88 545
Kernel 1666816 RIPON ROY 015 07 817
Kernel 1667031 RIPON ROY 012 61 683
Kernel 1667355 RIPON ROY 015 76 706
Kernel 1677498 RIPON ROY 011 75 731
Kernel 1711591 RIPON ROY 013 87 527
Kernel 1711595 RIPON ROY 018 82 251
Kernel 1724373 RIPON ROY 016 70 066
Kernel 1725052 RIPON ROY 016 16 483
Kernel 1725460 RIPON ROY 015 73 635
Kernel 1726842 RIPON ROY 015 28 004
Kernel 1730227 RIPON ROY RONI 013 04 255
Kernel 1731509 RIPON ROY 013 23 840
Kernel 1733558 RIPON ROY 014 81 288
Kernel 1733597 RIPON ROY 016 86 585
Kernel 1734875 RIPON ROY 014 64 012
Kernel 1736055 RIPON ROY 012 32 178
Kernel 1744791 RIPON ROY 013 30 275
Kernel 1751068 RIPON ROY 016 23 884
Kernel 1759617 RIPO